# **Data preparation**

This notebook contains the code to preprocess and convert lecture transcripts and course notes into embeddings for later use in the student copilot application

### **What's Semantic Kernel**

Semantic Kernel is an open-source SDK that lets you easily combine AI services like OpenAI, Azure OpenAI, and Hugging Face with conventional programming languages like C# and Python. By doing so, you can create AI apps that combine the best of both worlds.

![Copilot Stack](./imgs/mind-and-body-of-semantic-kernel.png)


Semantic Kernel has been engineered to allow developers to flexibly integrate AI services into their existing apps. To do so, Semantic Kernel provides a set of connectors that make it easy to add memories and models. In this way, Semantic Kernel is able to add a simulated "brain" to your app.

Additionally, Semantic Kernel makes it easy to add skills to your applications with AI plugins that allow you to interact with the real world. These plugins are composed of prompts and native functions that can respond to triggers and perform actions. In this way, plugins are like the "body" of your AI app.

Because of the extensibility Semantic Kernel provides with connectors and plugins, you can use it to orchestrate AI plugins from both OpenAI and Microsoft on top of nearly any model. For example, you can use Semantic Kernel to orchestrate plugins built for ChatGPT, Bing, and Microsoft 365 Copilot on top of models from OpenAI, Azure, or even Hugging Face.



using nuget to add Microsoft.SemanticKernel & Microsoft.SemanticKernel.Connectors.Memory.Qdrant library

In [2]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.Qdrant, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta3 Microsoft.SemanticKernel.Connectors.Memory.Qdrant, 1.0.0-beta3

In [3]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextEmbedding;

using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.Memory.Qdrant;

In [4]:
using System;
using System.IO;
using System.Text.Json;

### **Creating the kernel runtime environment**

By investigating the constructor of the Kernel class, you can see that you can configure multiple settings that are necessary to run both native and semantic functions. These include:

- The default AI service that will power your semantic functions.

- The template engine used to render prompt templates.

- The logger used to log messages from functions.

- The plugins available to be executed by the kernel

- Additional configuration used by the kernel via the KernelConfig class.

In [16]:
var AZURE_OPENAI_ENDPOINT = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var AZURE_OPENAI_KEY = Environment.GetEnvironmentVariable("AZURE_OPENAI_KEY");
var AZURE_OPENAI_EMBEDDING_DEPLOYMENT = Environment.GetEnvironmentVariable("AZURE_OPENAI_EMBEDDING_DEPLOYMENT");
var AZURE_OPENAI_CHAT_DEPLOYMENT = Environment.GetEnvironmentVariable("AZURE_OPENAI_CHAT_DEPLOYMENT");
var QDRANT_HOST = Environment.GetEnvironmentVariable("QDRANT_HOST");

In [6]:
IKernel kernel = new KernelBuilder().WithAzureChatCompletionService(AZURE_OPENAI_CHAT_DEPLOYMENT, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY).Build();

### **What is a plugin?**

To drive alignment across the industry, we've adopted the OpenAI plugin specification as the standard for plugins. This will help create an ecosystem of interoperable plugins that can be used across all of the major AI apps and services like ChatGPT, Bing, and Microsoft 365.


In [7]:
var pluginDirectory =  System.IO.Directory.GetCurrentDirectory() + "/Plugins";

Set plugin for File and Answer

In [8]:
var filePlugin = kernel.ImportSemanticFunctionsFromDirectory(pluginDirectory,"FilePlugin");
var answerPlugin = kernel.ImportSemanticFunctionsFromDirectory(pluginDirectory,"AnswerPlugin");

In [9]:
public class KB
{
    public string kb { get; set; }
    public string content { get; set; }
}

Get knowledge from transcripts

In [10]:
DirectoryInfo transcriptsFolder = new DirectoryInfo(@"./data/transcripts");    
FileInfo[] transcriptsFiles = transcriptsFolder.GetFiles();

In [11]:
IList<KB> kbList = new List<KB>();

foreach(var file in transcriptsFiles)
{
    string content = File.ReadAllText(file.FullName);
    var jsonResult = await kernel.RunAsync(content, filePlugin["Transcrips"]);
    var result = JsonSerializer.Deserialize<List<KB>>(jsonResult.ToString());
    foreach(var item in result)
    {
        kbList.Add(item);
    }
}

In [12]:
kbList

index value 0 Submission#8+KB kb Intro content in this video I want to give you a bit of Background by walking through the major milestones in the history of machine learning and artificial intelligence 1 Submission#8+KB kb Alan Turing and the Turing test content intelligence really begun in 1950s though it's based on mathematical and statistical developments over many centuries Alan Turing is credited with helping to lay the foundation for the concept of a machine they can think in his quest to Define machine intelligence he achieved a crucial Milestone by creating the Turing test in 1950. in this test an interrogator questions both a human and a computer and tries to determine which one is which if the interrogator cannot tell the difference then the computer can be considered intelligent 2 Submission#8+KB kb The Dartmouth Summer Research Project on AI content artificial intelligence was coined with a small group of scientists gathered at Dartmouth College in the U.S for an event called the summer research project on artificial intelligence this conference was the birth of the field of research we know as AI 3 Submission#8+KB kb The golden years of AI content the years from 1956 to 1974 are known as The Golden Ears of AI optimism ran high in the hope that AI could solve many problems in 1967 Marvin Minsky the co-founder of the mitai lab stated confidently and incorrectly that within a generation the problem of creating artificial intelligence will substantially be solved natural language processing research flourished search was refined and made more powerful and the concept of micro worlds was created where simple tasks were completed using plain language instructions research was well funded by government agencies advances were made in computation and algorithms and prototypes of intelligent machines were built some of these machines include shaky the robot who could maneuver and decide how to perform tasks intelligently Eliza and gnarly charabot that could converse with people and act as a primitive therapist Blocksworld an example of a micro world where blogs get could be stacked and sorted and decision-making experiments could be tested by the mid-1970s it had become apparent that the complexity of making intelligent machines had been understated and that its promise had been overblown compute power was too limited there was a lack of data to train and test AIS and there were questions around the ethics of introducing AI systems like the therapist Eliza into society funding dried up and confidence in the field slowed marking the beginning of what is called an AI winter 4 Submission#8+KB kb The AI winter content in the 1980s as computers became more powerful expert systems became more successful there was a Resurgence in optimism about AI as businesses found practical applications of these rule-based inference systems by the late 80s it was becoming apparent that expert systems had become too specialized and were unlikely to achieve machine intelligence the rise of personal computers also competed with these large specialized centralized systems this led to another chill in the AI field 5 Submission#8+KB kb Resurgence and fall of AI for expert systems content things began to change in the mid-1990s as compute and storage capabilities grew exponentially making it possible to process much larger data sets than ever before the rise of the internet and the popularity of smartphones both contributed to increasing amounts of data a new experiments in machine learning became possible throughout the 2000s significant advancements were made in computer vision and natural language processing by training machine learning models on Big Data 6 Submission#8+KB kb Growth in AI driven by more data and more powerful hardware content in the past decade compute power and the size of data sets have continued to grow and machine learning has become capable of solving even more problems as a result today machine learning touches almost every 

Get knowledge from notes

In [13]:
DirectoryInfo notesFolder = new DirectoryInfo(@"./data/notes");    
FileInfo[] notesFiles = notesFolder.GetFiles();

In [14]:

foreach(var file in notesFiles)
{
    string content = File.ReadAllText(file.FullName);
    var jsonResult = await kernel.RunAsync(content, filePlugin["Notes"]);
    var result = JsonSerializer.Deserialize<KB>(jsonResult.ToString());
    kbList.Add(result);
}

### **What's Embeddings**

An embedding is a relatively low-dimensional space into which you can translate high-dimensional vectors. Embeddings make it easier to do machine learning on large inputs like sparse vectors representing words. Ideally, an embedding captures some of the semantics of the input by placing semantically similar inputs close together in the embedding space. An embedding can be learned and reused across models.

**Note:** We use qdrant for this sample . please open terminal and run this code to start qdrant db

```bash

docker run -p 6333:6333 qdrant/qdrant

```

In [17]:
var qdrantMemoryBuilder = new MemoryBuilder();

var textEmbedding = new AzureTextEmbeddingGeneration(AZURE_OPENAI_EMBEDDING_DEPLOYMENT, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY);
qdrantMemoryBuilder.WithTextEmbeddingGeneration(textEmbedding);
qdrantMemoryBuilder.WithQdrantMemoryStore(QDRANT_HOST, 1536);

var builder = qdrantMemoryBuilder.Build();

save knowledge to vectordb

In [18]:
string MemoryCollectionName = "kb_collection";
int index = 1;


foreach(var item in kbList)
{
    await builder.SaveInformationAsync(MemoryCollectionName, id: "index"+(index++).ToString(), text: item.kb + " -  " + item.content);
}

### **Testing**

In [19]:
var searchResults =  builder.SearchAsync(MemoryCollectionName, "can you tell me what is different ML and AI", limit: 1, minRelevanceScore: 0.8);


In [20]:

await foreach (var item in searchResults)
{
    var answer = await kernel.RunAsync(item.Metadata.Text, answerPlugin["Summary"]);
    Console.WriteLine(answer.ToString());
}

going to focus on machine learning and its relationship to artificial intelligence. Machine learning is a branch of AI that uses algorithms to make decisions by learning from data. It is a popular and successful approach in AI. Deep learning, on the other hand, is a subset of machine learning that relies on neural networks to learn from data. In this course, we will primarily explore machine learning and its connection to artificial intelligence.


In [21]:
var searchResults =  builder.SearchAsync(MemoryCollectionName, "do you know 1956: Dartmouth Summer Research Project", limit: 1, minRelevanceScore: 0.7);


In [22]:
await foreach (var item in searchResults)
{
    var answer = await kernel.RunAsync(item.Metadata.Text, answerPlugin["Summary"]);
    Console.WriteLine(answer.ToString());
}

The Dartmouth Summer Research Project on AI, held at Dartmouth College in the U.S., marked the beginning of the field of artificial intelligence (AI). It brought together a small group of scientists who coined the term AI. This conference laid the foundation for the research and development of AI as we know it today.
